# Job for data transformation

Este notebook documenta a etapa de transformação de dados da camada Silver, que sucede a extração/ingestão de um relatório institucional de insucesso acadêmico exportado em formato Excel. O arquivo de origem, preservado na camada Bronze com abas por semestre, é lido integralmente e preparado para análise. As transformações incluem a remoção de linhas de rodapé não informativas, o descarte de atributos sem utilidade analítica, a tipagem explícita de identificadores textuais e contagens inteiras, a criação de variáveis derivadas que representem o semestre de oferta e o departamento de origem, bem como a concatenação das diferentes abas em um único conjunto de dados para análise longitudinal.

Do ponto de vista de modelagem, adota-se granularidade por disciplina e semestre, de modo que a combinação de Código e Semestre identifique de forma inequívoca cada registro. A derivação do Departamento a partir do prefixo do Código viabiliza recortes organizacionais consistentes e comparações entre áreas. A padronização de tipos com a explicitação de strings para identificadores e inteiros para contagens evita ambiguidades e reduz o risco de erros durante operações de junção e agregação. Além disso, a imutabilidade por partição temporal é preservada: cada semestre é processado individualmente e, em seguida, incorporado ao conjunto consolidado, o que facilita auditoria e reprodutibilidade.

A governança do processo é apoiada por verificações intermediárias sobre esquemas e contagens, garantindo rastreabilidade das alterações e coerência entre as fontes. Ao final, o resultado é materializado em um arquivo CSV padronizado, `lista-insucesso-processed.csv`, com separador ponto e vírgula, pronto para consumo pelas rotinas de análise e visualização. Reconhece-se, por fim, que a qualidade da análise depende da consistência do arquivo de origem e que investigações causais sobre o insucesso exigiriam variáveis adicionais não contempladas neste relatório; entretanto, a estruturação aqui adotada estabelece uma base sólida para monitoramento perene, comparabilidade entre semestres e expansão futura do escopo analítico.

## Imports

### import the required libs and create the spark session

In [ ]:
import pandas as pd
import findspark
findspark.init()

from functools import reduce
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as F

try:
    spark.stop()
    print("Previous Spark session stopped.")
except NameError:
    print("No active Spark session found.")

# Build the SparkSession
spark = SparkSession.builder \
    .appName("Spark 4 Test App") \
    .getOrCreate()

print(f"✅ Spark Version: {spark.version}")

No active Spark session found.


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/10 16:31:57 WARN Utils: Your hostname, dartmol203, resolves to a loopback address: 127.0.1.1; using 192.168.0.17 instead (on interface wlp2s0)
25/10/10 16:31:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/10 16:31:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


✅ Spark Version: 4.0.1


## Read data

### Read the data from Excel sheets with pandas

In [ ]:
dataframes_all_sheets = pd.read_excel("../bronze/Relatorio-Lista-Insucesso.xlsx", sheet_name=None)
dataframes_all_sheets_automotiva = pd.read_excel("../bronze/Relatorio-Lista-Insucesso-Automotiva.xlsx", sheet_name=None)

### Convert the dataframes read from pandas to Spark

In [ ]:
spark_dfs_software = {}
for sheet_name, pd_df in dataframes_all_sheets.items():
    print(f"Converting sheet: {sheet_name}")
    spark_dfs_software[sheet_name] = spark.createDataFrame(pd_df)

spark_dfs_automotiva = {}
for sheet_name, pd_df in dataframes_all_sheets_automotiva.items():
    print(f"Converting sheet: {sheet_name}")
    spark_dfs_automotiva[sheet_name] = spark.createDataFrame(pd_df)

Converting sheet: 2025.1
Converting sheet: 2024.2
Converting sheet: 2024.1
Converting sheet: 2023.2
Converting sheet: 2025.1
Converting sheet: 2024.2
Converting sheet: 2024.1
Converting sheet: 2023.2


### Show some info about one sheet ['2025.1']

In [ ]:
spark_dfs_software['2025.1'].show()

+-------+--------------------+----+------+---------+-------------+-----------------+----------------+-----------------+-------------------------+------------------------+------------+---------------+
| Código|                Nome|Pólo|Turmas|Discentes|Cancelamentos|Reprovações Média|Reprovações Nota|Reprovações Falta|Reprovações Média e Falta|Reprovações Nota e Falta|Trancamentos|Total Insucesso|
+-------+--------------------+----+------+---------+-------------+-----------------+----------------+-----------------+-------------------------+------------------------+------------+---------------+
|MAT0039|      ALGEBRA LINEAR| NaN|     1|        1|            0|                0|               0|                0|                        0|                       0|           1|              1|
|CIC0004|ALGORITMOS E PROG...| NaN|     5|       38|            3|                5|               0|                2|                        0|                       0|           1|             11|


In [ ]:
spark_dfs_software['2025.1'].printSchema()

root
 |-- Código: string (nullable = true)
 |-- Nome: string (nullable = true)
 |-- Pólo: double (nullable = true)
 |-- Turmas: long (nullable = true)
 |-- Discentes: long (nullable = true)
 |-- Cancelamentos: long (nullable = true)
 |-- Reprovações Média: long (nullable = true)
 |-- Reprovações Nota: long (nullable = true)
 |-- Reprovações Falta: long (nullable = true)
 |-- Reprovações Média e Falta: long (nullable = true)
 |-- Reprovações Nota e Falta: long (nullable = true)
 |-- Trancamentos: long (nullable = true)
 |-- Total Insucesso: long (nullable = true)



In [ ]:
spark_dfs_software['2025.1'].describe().show()

25/10/10 16:32:19 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-------+--------------------+----+------------------+-----------------+------------------+-----------------+----------------+-----------------+-------------------------+------------------------+-----------------+------------------+
|summary| Código|                Nome|Pólo|            Turmas|        Discentes|     Cancelamentos|Reprovações Média|Reprovações Nota|Reprovações Falta|Reprovações Média e Falta|Reprovações Nota e Falta|     Trancamentos|   Total Insucesso|
+-------+-------+--------------------+----+------------------+-----------------+------------------+-----------------+----------------+-----------------+-------------------------+------------------------+-----------------+------------------+
|  count|    150|                 150| 150|               150|              150|               150|              150|             150|              150|                      150|                     150|              150|               150|
|   mean|   NULL|                 Na

## Transform data

### Process first software, then automotiva

#### Create an array with the semester codes

In [ ]:
semesters = ['2023.2', '2024.1','2024.2','2025.1']


#### Do the transformation on the software dataframes

In [ ]:

transformed_dfs = [
    spark_dfs_software[semester]
        .drop('Pólo')
        .withColumn("Semestre", F.lit(semester))
        .withColumn("Departamento", F.substring("Código", 1, 3))
        .withColumn("Curso", F.lit("Software"))
        .filter(F.col("Código") != "Total:")
        .dropna()
    for semester in semesters
]
    
if transformed_dfs:
    final_spark_dataframe = reduce(DataFrame.unionByName, transformed_dfs)
    final_spark_dataframe.show()
else:
    print("No DataFrames to union.")
    

+-------+--------------------+------+---------+-------------+-----------------+----------------+-----------------+-------------------------+------------------------+------------+---------------+--------+------------+--------+
| Código|                Nome|Turmas|Discentes|Cancelamentos|Reprovações Média|Reprovações Nota|Reprovações Falta|Reprovações Média e Falta|Reprovações Nota e Falta|Trancamentos|Total Insucesso|Semestre|Departamento|   Curso|
+-------+--------------------+------+---------+-------------+-----------------+----------------+-----------------+-------------------------+------------------------+------------+---------------+--------+------------+--------+
|ADM0014|ADMINISTRAÇÃO PÚB...|     1|        1|            0|                0|               0|                0|                        0|                       0|           0|              0|  2023.2|         ADM|Software|
|FGA0038|AERODINÂMICA DE S...|     1|        1|            0|                0|               0|

#### Do the transformations on Automotiva dataframes

In [ ]:

transformed_dfs_automotiva = [
    spark_dfs_automotiva[semester]
        .drop('Pólo')
        .withColumn("Semestre", F.lit(semester))
        .withColumn("Departamento", F.substring("Código", 1, 3))
        .withColumn("Curso", F.lit("Automotiva"))
        .filter(F.col("Código") != "Total:")
        .dropna()
    for semester in semesters
]
    
if transformed_dfs_automotiva:
    final_spark_dataframe = reduce(DataFrame.unionByName, [final_spark_dataframe] + transformed_dfs_automotiva)
    final_spark_dataframe.show()
else:
    print("No DataFrames to union.")
    

+-------+--------------------+------+---------+-------------+-----------------+----------------+-----------------+-------------------------+------------------------+------------+---------------+--------+------------+--------+
| Código|                Nome|Turmas|Discentes|Cancelamentos|Reprovações Média|Reprovações Nota|Reprovações Falta|Reprovações Média e Falta|Reprovações Nota e Falta|Trancamentos|Total Insucesso|Semestre|Departamento|   Curso|
+-------+--------------------+------+---------+-------------+-----------------+----------------+-----------------+-------------------------+------------------------+------------+---------------+--------+------------+--------+
|ADM0014|ADMINISTRAÇÃO PÚB...|     1|        1|            0|                0|               0|                0|                        0|                       0|           0|              0|  2023.2|         ADM|Software|
|FGA0038|AERODINÂMICA DE S...|     1|        1|            0|                0|               0|

#### transform semester column to the format year-semester

In [ ]:
final_spark_dataframe = final_spark_dataframe.withColumn(
    "Semestre", 
    F.regexp_replace("Semestre", "\\.", "-")
)

### Final dataframe

#### Show statistics of the final data

In [ ]:
final_spark_dataframe.printSchema()

root
 |-- Código: string (nullable = true)
 |-- Nome: string (nullable = true)
 |-- Turmas: long (nullable = true)
 |-- Discentes: long (nullable = true)
 |-- Cancelamentos: long (nullable = true)
 |-- Reprovações Média: long (nullable = true)
 |-- Reprovações Nota: long (nullable = true)
 |-- Reprovações Falta: long (nullable = true)
 |-- Reprovações Média e Falta: long (nullable = true)
 |-- Reprovações Nota e Falta: long (nullable = true)
 |-- Trancamentos: long (nullable = true)
 |-- Total Insucesso: long (nullable = true)
 |-- Semestre: string (nullable = false)
 |-- Departamento: string (nullable = true)
 |-- Curso: string (nullable = false)



In [ ]:
final_spark_dataframe.describe().show()

+-------+-------+----------------+------------------+-----------------+------------------+------------------+----------------+------------------+-------------------------+------------------------+------------------+------------------+--------+------------+----------+
|summary| Código|            Nome|            Turmas|        Discentes|     Cancelamentos| Reprovações Média|Reprovações Nota| Reprovações Falta|Reprovações Média e Falta|Reprovações Nota e Falta|      Trancamentos|   Total Insucesso|Semestre|Departamento|     Curso|
+-------+-------+----------------+------------------+-----------------+------------------+------------------+----------------+------------------+-------------------------+------------------------+------------------+------------------+--------+------------+----------+
|  count|    917|             917|               917|              917|               917|               917|             917|               917|                      917|                     917|

In [ ]:
final_spark_dataframe

DataFrame[Código: string, Nome: string, Turmas: bigint, Discentes: bigint, Cancelamentos: bigint, Reprovações Média: bigint, Reprovações Nota: bigint, Reprovações Falta: bigint, Reprovações Média e Falta: bigint, Reprovações Nota e Falta: bigint, Trancamentos: bigint, Total Insucesso: bigint, Semestre: string, Departamento: string, Curso: string]

In [ ]:
final_spark_dataframe.show()

+-------+--------------------+------+---------+-------------+-----------------+----------------+-----------------+-------------------------+------------------------+------------+---------------+--------+------------+--------+
| Código|                Nome|Turmas|Discentes|Cancelamentos|Reprovações Média|Reprovações Nota|Reprovações Falta|Reprovações Média e Falta|Reprovações Nota e Falta|Trancamentos|Total Insucesso|Semestre|Departamento|   Curso|
+-------+--------------------+------+---------+-------------+-----------------+----------------+-----------------+-------------------------+------------------------+------------+---------------+--------+------------+--------+
|ADM0014|ADMINISTRAÇÃO PÚB...|     1|        1|            0|                0|               0|                0|                        0|                       0|           0|              0|  2023-2|         ADM|Software|
|FGA0038|AERODINÂMICA DE S...|     1|        1|            0|                0|               0|

#### Export dataframe to .csv

In [ ]:
final_spark_dataframe.coalesce(1) \
    .write \
    .option("header", "true") \
    .option("delimiter", ";") \
    .mode("overwrite") \
    .csv("lista-insucesso-processed-spark")